# 카카오 API : 네비게이션 검색

- 카카오 API 테스트
  - 앱 ID : 
  - 네이티브 앱 키 : 
  - REST API 키 : 
  - JavaScript 키 : 
  - Admin 키 : 
- 키워드 검색
  - 질의어를 통해 장소 검색 결과 반환
  - 네비게이션 길안내 결과 반환

ex)
- origin : 33.248678, 126.56734
- destiantion : 33.291055, 126.461936
- 한라산 : 33.361204769954284, 126.53111502985787
---
- 경로 탐색 실패하는 경우 발생 : **출발지, 도착지 주변에 도로가 없는 경우**

## 1. 경로 검색

In [ ]:
import json
import requests
# REST 키
rest_api_key = ''
# 헤더
headers = {"Authorization" : "KakaoAK {}".format(rest_api_key)}
# 파라미터
url1 = "https://apis-navi.kakaomobility.com/v1/directions?origin=126.56734,33.248678&destination=126.461936,33.291055&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"
url2 = "https://apis-navi.kakaomobility.com/v1/directions?origin=126.56734,33.248678&destination=126.53111502985787,33.361204769954284&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"

# GET을 이용하여 획득
res1 = requests.get(url1, headers=headers)
# Json을 이용하여 해제
doc1 = json.loads(res1.text)

res2 = requests.get(url2, headers=headers)
doc2 = json.loads(res2.text)

# 200일 경우 정상
res1.status_code, res2.status_code

In [ ]:
# 결과(Dict형으로 표현)
print(doc1['routes'][0]['result_code'], doc1['routes'][0]['result_msg'],)
doc1['routes'][0]['summary']

0 길찾기 성공


{'origin': {'name': '', 'x': 126.56733334860414, 'y': 33.24867189296654},
 'destination': {'name': '', 'x': 126.46192692717538, 'y': 33.29105000445024},
 'waypoints': [],
 'priority': 'RECOMMEND',
 'bound': {'min_x': 126.46225482364076,
  'min_y': 33.24724561321495,
  'max_x': 126.56944343869672,
  'max_y': 33.29365953466207},
 'fare': {'taxi': 13800, 'toll': 0},
 'distance': 15004,
 'duration': 1522}

In [ ]:
doc2

{'trans_id': '05b61c85169846c38177fe407d6ded62',
 'routes': [{'result_code': 103, 'result_msg': '도착지점 주변의 도로를 탐색할 수 없음'}]}

In [ ]:
vertexes = []
for road in doc1['routes'][0]['sections'][0]['roads']:
    test_v = road['vertexes']
    for i in range(0, len(test_v), 2):
        vertexes.append((test_v[i+1], test_v[i]))

In [ ]:
vertexes[:10]

[(33.24864534401754, 126.567376703514),
 (33.24864534401754, 126.567376703514),
 (33.248590268564406, 126.56729175890251),
 (33.248590268564406, 126.56729175890251),
 (33.24851740006151, 126.56644524610292),
 (33.24851740006151, 126.56644524610292),
 (33.24848565644762, 126.56603800926246),
 (33.24848565644762, 126.56603800926246),
 (33.249097401159204, 126.56592071290774),
 (33.249097401159204, 126.56592071290774)]

## 2. 경로 표현

In [ ]:
import folium
import pandas as pd

origin = doc1['routes'][0]['summary']['origin']
destination = doc1['routes'][0]['summary']['destination']

map = folium.Map(location=[origin['y'], origin['x']], zoom_start=9)
folium.Marker(location=[origin['y'], origin['x']], popup='origin').add_to(map)
folium.Marker(location=[destination['y'], destination['x']], popup='destination').add_to(map)
folium.PolyLine(vertexes, color='red').add_to(map)
map